In [1]:
from preprocess import get_img_paths, prep_data
import numpy as np

### Get image file paths

In [2]:
fractured, nfractured = get_img_paths('train')

### Preprocess data for train

In [3]:
x_1, y_1 = prep_data(fractured, 1)
x_0, y_0 = prep_data(nfractured, 0)
x_train = np.concatenate([x_0,x_1], axis=0)
x_train = np.expand_dims(x_train, axis=-1)
y_train = np.concatenate([y_0,y_1], axis=0)
x_train.shape, y_train.shape

((8538, 224, 224, 1), (8538, 1))

### Preprocess data for validation

In [4]:
val_fractured, val_nfractured = get_img_paths('val')
x_1, y_1 = prep_data(val_fractured, 1)
x_0, y_0 = prep_data(val_nfractured, 0)
x_val = np.concatenate([x_0,x_1], axis=0)
x_val = np.expand_dims(x_val, axis=-1)
y_val = np.concatenate([y_0,y_1], axis=0)
x_val.shape, y_val.shape

((582, 224, 224, 1), (582, 1))

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, LeakyReLU, Reshape, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
from tensorflow.keras import layers, models

### Autoencoder architecture (Non-linear dimension reduction)

In [6]:
input_shape = (224, 224, 1)

# Encoder
encoder = Sequential([
    Flatten(input_shape=input_shape),  # Flatten layer to convert input to 1D
    
    Dense(255),
    LeakyReLU(alpha=0.1),
    
    Dense(9),
    LeakyReLU(alpha=0.1),
], name="encoder")

# Decoder
decoder = Sequential([
    Dense(255, input_shape=(9,)),
    LeakyReLU(alpha=0.1),
    
    Dense(224 * 224),
    LeakyReLU(alpha=0.1),
    
    Reshape((224, 224, 1))
], name="decoder")

# Combine encoder and decoder to create the autoencoder
autoencoder_input = Input(shape=input_shape)
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = Model(autoencoder_input, decoded)

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 encoder (Sequential)        (None, 9)                 12797439  
                                                                 
 decoder (Sequential)        (None, 224, 224, 1)       12847606  
                                                                 
Total params: 25645045 (97.83 MB)
Trainable params: 25645045 (97.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### MSE Loss on Auto encoder-decoder

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
autoencoder.fit(x_train, x_train, epochs=100, batch_size=64, validation_data=(x_val, x_val), callbacks=[early_stopping])

Epoch 1/100
134/134 [==============================] - 42s 309ms/step - loss: 2867.2366 - val_loss: 1256.2069
Epoch 2/100
134/134 [==============================] - 43s 320ms/step - loss: 1104.8633 - val_loss: 990.6733
Epoch 3/100
134/134 [==============================] - 43s 321ms/step - loss: 924.3425 - val_loss: 828.9111
Epoch 4/100
134/134 [==============================] - 42s 310ms/step - loss: 810.5392 - val_loss: 752.7122
Epoch 5/100
134/134 [==============================] - 40s 301ms/step - loss: 758.1308 - val_loss: 715.9968
Epoch 6/100
134/134 [==============================] - 40s 297ms/step - loss: 716.0281 - val_loss: 683.0338
Epoch 7/100
134/134 [==============================] - 40s 298ms/step - loss: 685.9343 - val_loss: 660.0860
Epoch 8/100
134/134 [==============================] - 40s 296ms/step - loss: 671.4184 - val_loss: 638.3483
Epoch 9/100
134/134 [==============================] - 40s 301ms/step - loss: 645.3823 - val_loss: 618.5173
Epoch 10/100
134/134 [===

### Pipe PCA (linear dimension reduction) with decoder

In [40]:
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [41]:
x_train_flatten = x_train.reshape((x_train.shape[0], -1))
x_val_flatten = x_val.reshape((x_val.shape[0], -1))

pca = PCA(n_components=9)
x_train_pca = pca.fit_transform(x_train_flatten)
x_val_pca = pca.transform(x_val_flatten)

print("x_train_pca shape:", x_train_pca.shape)
print("x_val_pca shape:", x_val_pca.shape)

x_train_pca shape: (8538, 9)
x_val_pca shape: (582, 9)


### MSE Loss on PCA Inverse Transform

In [49]:
x_train_reconstructed = pca.inverse_transform(x_train_pca)
x_val_reconstructed = pca.inverse_transform(x_val_pca)

mse_train = mean_squared_error(x_train.reshape(x_train.shape[0],-1), x_train_reconstructed)
mse_val = mean_squared_error(x_val.reshape(x_val.shape[0], -1), x_val_reconstructed)

print(f"MSE Loss on Training Data: {mse_train}")
print(f"MSE Loss on Validation Data: {mse_val}")

MSE Loss on Training Data: 852.550436347354
MSE Loss on Validation Data: 830.2604379859235


In [9]:
# Define the decoder
pca_decoder = Sequential([
    Dense(120, input_shape=(9,)),
    LeakyReLU(alpha=0.1),
    
    Dense(224 * 224),
    LeakyReLU(alpha=0.1),
    
    Reshape((224, 224, 1))
], name="pca_decoder")

pca_decoder.compile(optimizer='adam', loss='mse')
pca_decoder.summary()

Model: "pca_decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 120)               1200      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 120)               0         
                                                                 
 dense_5 (Dense)             (None, 50176)             6071296   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 50176)             0         
                                                                 
 reshape_1 (Reshape)         (None, 224, 224, 1)       0         
                                                                 
Total params: 6072496 (23.16 MB)
Trainable params: 6072496 (23.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### MSE Loss on PCA + Decoder

In [11]:
pca_decoder.fit(x_train_pca, x_train, epochs=100, batch_size=64, validation_data=(x_val_pca, x_val), callbacks=[early_stopping])

Epoch 1/100
134/134 [==============================] - 17s 119ms/step - loss: 1146.5815 - val_loss: 877.1307
Epoch 2/100
134/134 [==============================] - 17s 129ms/step - loss: 844.2220 - val_loss: 799.5322
Epoch 3/100
134/134 [==============================] - 14s 108ms/step - loss: 793.8442 - val_loss: 782.5869
Epoch 4/100
134/134 [==============================] - 14s 105ms/step - loss: 768.4728 - val_loss: 745.5298
Epoch 5/100
134/134 [==============================] - 14s 108ms/step - loss: 745.1210 - val_loss: 725.2252
Epoch 6/100
134/134 [==============================] - 14s 107ms/step - loss: 725.4292 - val_loss: 709.6722
Epoch 7/100
134/134 [==============================] - 14s 105ms/step - loss: 710.7993 - val_loss: 704.0375
Epoch 8/100
134/134 [==============================] - 15s 111ms/step - loss: 703.2197 - val_loss: 691.8526
Epoch 9/100
134/134 [==============================] - 15s 114ms/step - loss: 692.1844 - val_loss: 678.1227
Epoch 10/100
134/134 [=====

### Classify images with encoder neural network

In [12]:
classifier_layer = Sequential([
    Dense(1, activation='sigmoid', input_shape=(9,))
], name="nn_classifier")

encoded_input = Input(shape=input_shape)
encoded_output = encoder(encoded_input)
classification_output = classifier_layer(encoded_output)
nn_classifier = Model(encoded_input, classification_output)

nn_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

nn_classifier.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/100
267/267 [==============================] - 42s 154ms/step - loss: 30.2008 - accuracy: 0.7467 - val_loss: 4.8352 - val_accuracy: 0.8591
Epoch 2/100
267/267 [==============================] - 41s 154ms/step - loss: 6.6809 - accuracy: 0.8572 - val_loss: 20.7972 - val_accuracy: 0.5945
Epoch 3/100
267/267 [==============================] - 40s 149ms/step - loss: 4.0326 - accuracy: 0.9063 - val_loss: 1.3792 - val_accuracy: 0.9536
Epoch 4/100
267/267 [==============================] - 43s 160ms/step - loss: 2.1450 - accuracy: 0.9352 - val_loss: 2.7547 - val_accuracy: 0.9175
Epoch 5/100
267/267 [==============================] - 40s 150ms/step - loss: 1.6394 - accuracy: 0.9442 - val_loss: 1.3954 - val_accuracy: 0.9399
Epoch 6/100
267/267 [==============================] - 40s 151ms/step - loss: 1.3939 - accuracy: 0.9512 - val_loss: 0.7956 - val_accuracy: 0.9691
Epoch 7/100
267/267 [==============================] - 42s 159ms/step - loss: 0.6926 - accuracy: 0.9665 - val_loss: 0.2634

### PCA variance at first 9 components

In [20]:
sum(pca.explained_variance_ratio_)

0.6026426973329506

In [33]:
pca_classifier = Sequential([
    Input(shape=(9,)),
    Dense(1, activation='sigmoid')
], name="pca_classifier")

pca_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
pca_classifier.fit(x_train_pca, y_train, epochs=100, batch_size=64, validation_data=(x_val_pca, y_val), callbacks=[early_stopping])

Epoch 1/100
134/134 [==============================] - 1s 4ms/step - loss: 1104.1412 - accuracy: 0.5080 - val_loss: 870.3793 - val_accuracy: 0.5430
Epoch 2/100
134/134 [==============================] - 0s 2ms/step - loss: 741.9011 - accuracy: 0.5231 - val_loss: 545.1534 - val_accuracy: 0.5739
Epoch 3/100
134/134 [==============================] - 0s 2ms/step - loss: 426.5639 - accuracy: 0.5538 - val_loss: 265.5727 - val_accuracy: 0.5842
Epoch 4/100
134/134 [==============================] - 0s 2ms/step - loss: 159.9816 - accuracy: 0.5814 - val_loss: 55.7009 - val_accuracy: 0.6134
Epoch 5/100
134/134 [==============================] - 0s 2ms/step - loss: 12.6731 - accuracy: 0.5704 - val_loss: 0.7002 - val_accuracy: 0.6375
Epoch 6/100
134/134 [==============================] - 0s 2ms/step - loss: 0.8961 - accuracy: 0.5622 - val_loss: 0.8758 - val_accuracy: 0.5155
Epoch 7/100
134/134 [==============================] - 0s 2ms/step - loss: 0.8337 - accuracy: 0.5743 - val_loss: 0.8365 - val

In [24]:
test_fractured, test_nfractured = get_img_paths('test')
x_1, y_1 = prep_data(test_fractured, 1)
x_0, y_0 = prep_data(test_nfractured, 0)
x_test = np.concatenate([x_0,x_1], axis=0)
x_test = np.expand_dims(x_test, axis=-1)
y_test = np.concatenate([y_0,y_1], axis=0)
x_test.shape, y_test.shape

((392, 224, 224, 1), (392, 1))

In [37]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

def make_cx(y_true, y_pred, model_name = ''):
    cm = confusion_matrix(y_true, y_pred)

    fig = px.imshow(
        cm,
        text_auto=True,
        labels=dict(x="Predicted Label", y="True Label", color="Count"),
        x=['Predicted fractured', 'Predicted nonfractured'],
        y=['Actual fractured', 'Actual nonfractured'],
        title="Confusion Matrix: " + model_name
    )

    return fig.show()

In [38]:
make_cx(y_test, nn_classifier.predict(x_test).astype(np.int32), 'Autoencoder')

13/13 [==============================] - 0s 18ms/step


In [39]:
make_cx(y_test, pca_classifier.predict(pca.transform(x_test.reshape(x_test.shape[0],-1))).astype(np.int32), 'PCA - k = 9')

13/13 [==============================] - 0s 2ms/step
